<a href="https://colab.research.google.com/github/Ataerman/LLM_RAG-AI_AGENT/blob/main/RAG_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
!pip install langchain langchain-openai langchain-cohere langchain-community \
sentence-transformers faiss-cpu \
datasets accelerate \
tiktoken \
cohere


In [91]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
openai_api_key=userdata.get('openai')
cohere_api_key=userdata.get('cohere')
openai_llm=ChatOpenAI(model='gpt-3.5-turbo', #creating llm model (using chatgpt 3.5 with api)
                      openai_api_key=openai_api_key,
                      temperature=0.3)


In [92]:
text = """Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics.
Since its premiere, Interstellar gained a cult following, and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades."""

In [93]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

doc = Document(page_content=text)
# 🔹 Text Splitter: Break the text into smaller chunks with overlap
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
docs = splitter.split_documents([doc])
print(docs[0].page_content)

# 🔹 Embedding Model: Convert chunks into numerical vector representations
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# 🔹 FAISS Vectorstore: Store and index the embeddings for similarity search
vectorstore = FAISS.from_documents(docs, embeddings)

Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher


In [94]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain.chains import RetrievalQA

# 🔹 Retriever: First retrieve top 10 similar chunks with FAISS
# 🔹 Then rerank them using Cohere Rerank API to improve relevance
retriever = ContextualCompressionRetriever(
    base_compressor=CohereRerank(
        cohere_api_key=cohere_api_key,
        model='rerank-multilingual-v2.0',
        top_n=3  # Return top 3 reranked chunks
    ),
    base_retriever=vectorstore.as_retriever(search_kwargs={'k': 10})
)
qa_chain=RetrievalQA.from_chain_type(
    llm=openai_llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True)

In [95]:
qa_chain.invoke("Who directed the movie Interstellar?")


{'query': 'Who directed the movie Interstellar?',
 'result': 'The movie Interstellar was directed by Christopher Nolan.',
 'source_documents': [Document(metadata={'relevance_score': 0.9999987}, page_content='Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher'),
  Document(metadata={'relevance_score': 0.8020909}, page_content='co-written, directed, and produced by Christopher Nolan.'),
  Document(metadata={'relevance_score': 0.31268474}, page_content='Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script')]}